## Fake News Classifier Using Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [6]:
import pandas as pd
!pip install kaggle



In [1]:
!pip install gensim==4.3.1

  Obtaining dependency information for gensim==4.3.1 from https://files.pythonhosted.org/packages/b2/cd/de59ef20e82402a320b574cbde40d559e916d2436ea9871b3e03efbfa09f/gensim-4.3.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/23.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/23.9 MB 991.0 kB/s eta 0:00:25
   ---------------------------------------- 0.1/23.9 MB 1.6 MB/s eta 0:00:15
   ---------------------------------------- 0.3/23.9 MB 2.0 MB/s eta 0:00:13
    --------------------------------------- 0.4/23.9 MB 2.3 MB/s eta 0:00:11
    --------------------------------------- 0.5/23.9 MB 2.6 MB/s eta 0:00:10
   - -------------------------------------- 0.7/23.9 MB 2.8 MB/s eta 0:00:09
   - -------------------------------------- 1.0/23.9 MB 3.2 MB/s eta 0:00:08
   -- ------------------------------------- 1.4/23.9 MB 3.8 MB/s eta 0:00:06
   -- ------------------------------------- 1.6/23.9 MB 4.2 MB/s eta 0:00:06
   --- ---------------

In [3]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [4]:
!kaggle competitions download -c fake-news

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Anaconda3\Scripts\kaggle.exe\__main__.py", line 4, in <module>
  File "C:\Anaconda3\Lib\site-packages\kaggle\__init__.py", line 7, in <module>
    api.authenticate()
  File "C:\Anaconda3\Lib\site-packages\kaggle\api\kaggle_api_extended.py", line 398, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in C:\Users\u013709\.kaggle. Or use the environment method.


In [4]:
import os
os.getcwd()

'C:\\Users\\u013709\\FakeNews\\Fake-New-LSTM'

In [7]:
df=pd.read_csv(r'C:\Users\u013709\Kaggle\FakeNews\train.csv')

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
###Drop Nan Values
df=df.dropna()


In [10]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [11]:
## Get the Dependent features
y=df['label']

In [12]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [13]:
X.shape

(18285, 4)

In [14]:
y.shape

(18285,)

In [15]:
import tensorflow as tf

In [16]:
tf.__version__

'2.16.1'

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [18]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [19]:
messages=X.copy()

In [20]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [21]:
messages.reset_index(inplace=True)

In [22]:
import nltk
import re
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\u013709\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')  # Download WordNet (lexical database)
nltk.download('omw-1.4')
nltk.download('punkt')  # For sentence tokenization
nltk.download('averaged_perceptron_tagger')  # For POS tagging
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\u013709\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\u013709\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\u013709\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\u013709\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [25]:

lemmatizer = WordNetLemmatizer()

#take mapping
def get_wordnet_pos(treebank_tag):
    """Maps treebank POS tags to WordNet POS tags."""
    if treebank_tag.startswith('J'):
        return 'a'  # Adjective
    elif treebank_tag.startswith('V'):
        return 'v'  # Verb
    elif treebank_tag.startswith('N'):
        return 'n'  # Noun
    elif treebank_tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return ''

def lemmatiser_word(sentence):
    lemma =[]
    tokens = word_tokenize(sentence)  # Tokenize the sentence into words
    tagged_words = pos_tag(tokens)    # Tag each word with its POS
    for word,tag in tagged_words:
        if not word in stopwords.words('english'):
            pos_tag_simple=get_wordnet_pos(tag)
            if pos_tag_simple:
                lemma.append(lemmatizer.lemmatize(word, pos=pos_tag_simple))
            else:
                lemma.append(lemmatizer.lemmatize(word))
            
    return " ".join(lemma)

In [26]:
print(lemmatiser_word(messages['title'][20]))

Abortion Pill Orders Rise 7 Latin American Nations Zika Alert - The New York Times


In [27]:
### Dataset Preprocessingfrom nltk.stem import WordNetLemmatizer


from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    #review = review.split()
    
    #review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    #review = ' '.join(review)
    review = lemmatiser_word(review)
    
    corpus.append(review)

In [28]:
corpus

['house dem aide even see comey letter jason chaffetz tweet',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fire',
 'civilian kill single u airstrike identify',
 'iranian woman jail fictional unpublished story woman stone death adultery',
 'jackie mason hollywood would love trump bomb north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organize action partner soros link indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor pay million release 

#onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr=encoding_tfidf(corpus)
pad_length=20

array=[]
for i in range(len(corpus)):
    array.append(drop_zeroes(onehot_repr[i]))
    
pad_sequences(array,padding='pre',maxlen=pad_length)    

array.shape

In [32]:
def drop_zeroes(array):
    pad_length = 20
    return_array=[]
    
    for i in array:
        if i == 0 :
            pass
        else :
            return_array.append(i)
    return return_array



In [33]:
print(drop_zeroes([0,0,0,0,0,0,2, 2 , 3 ,6,0,0,0,0,0,0,0]))

[2, 2, 3, 6]


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def encoding_tfidf(corpus):
    # Vectorize the text data using TF-IDF
    global  voc_size
    vectorizer = TfidfVectorizer()
    data = vectorizer.fit_transform(corpus)
    # Get the feature names (vocabulary)
    feature_names = vectorizer.get_feature_names_out()
    word_to_index = {word: index for index, word in enumerate(feature_names)}
    voc_size =len(feature_names)
    #Numerical Representation (Using TF-IDF Values)
    sequences = []
    for row in data.toarray():
        sentence_indices = [word_to_index[feature_names[i]] for i in np.nonzero(row)[0]]
        sequences.append(sentence_indices)
    return sequences


In [35]:
encoding_tfidf(corpus)

[[293, 2314, 2742, 3641, 4872, 6736, 7460, 8193, 12710, 14769],
 [1426, 1785, 2084, 2592, 5428, 6559, 15830],
 [5319, 5849, 9042, 14704],
 [318, 2523, 6863, 7787, 13091],
 [193, 3481, 5259, 7337, 7431, 13687, 13703, 15024, 15830],
 [1223,
  1631,
  1785,
  4924,
  6638,
  7418,
  7883,
  7951,
  8425,
  8750,
  9793,
  14553,
  14676,
  15312,
  15874],
 [1355, 5599, 6273, 9663, 9766, 10419, 11106, 13284, 14385, 15765, 15988],
 [857, 1052, 2340, 3116, 9663, 10788, 12366, 14385, 14676, 14822, 15988],
 [122, 259, 4051, 7067, 8293, 9901, 10115, 10416, 13358, 14676],
 [1250, 2243, 2738, 6740, 7364, 10195, 11616, 13126],
 [733, 1198, 3992, 6800, 9066, 9562, 10683, 11950, 12368, 12697, 14599],
 [8293, 9974, 12366, 12710, 14676],
 [1558, 5520, 6011, 8885, 10297, 13282, 14645, 15587, 15962],
 [742, 5305, 6648, 8094, 8569, 9663, 13280, 13817, 14385, 15988],
 [45, 2424, 5119, 9663, 13641, 14385, 15624, 15988],
 [76, 580, 778, 3374, 4174, 4883, 9076, 10479, 10758, 11828],
 [2601, 5162, 6559],
 [1

In [36]:
voc_size

16084

### Embedding Representation

In [37]:
sent_length=20
#embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs=pad_sequences(encoding_tfidf(corpus),padding='pre',maxlen=sent_length)
pd.DataFrame(embedded_docs,columns=[f'column{column}' for column in range(1,21)])

,column1,column2,column3,column4,column5,column6,column7,column8,column9,column10,column11,column12,column13,column14,column15,column16,column17,column18,column19,column20
0,0,0,0,0,0,0,0,0,0,0,293,2314,2742,3641,4872,6736,7460,8193,12710,14769
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1426,1785,2084,2592,5428,6559,15830
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5319,5849,9042,14704
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,318,2523,6863,7787,13091
4,0,0,0,0,0,0,0,0,0,0,0,193,3481,5259,7337,7431,13687,13703,15024,15830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2425,10983,11558,13922,14676,15683
18281,0,0,0,0,0,0,0,0,0,0,0,0,0,8773,9663,9954,10818,12577,14385,15988
18282,0,0,0,0,0,0,0,0,0,0,694,1241,6764,8514,9663,11653,12542,14095,14385,15988
18283,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1107,4938,6626,9544,10366,12366


In [38]:
embedded_docs[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,   293,  2314,  2742,  3641,  4872,  6736,  7460,  8193,
       12710, 14769])

In [39]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

C:\Anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [40]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.5))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [41]:
len(embedded_docs),y.shape

(18285, (18285,))

In [42]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [43]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [45]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 53ms/step - accuracy: 0.7647 - loss: 0.4267 - val_accuracy: 0.9319 - val_loss: 0.1750
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.9681 - loss: 0.0950 - val_accuracy: 0.9266 - val_loss: 0.1945
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - accuracy: 0.9886 - loss: 0.0411 - val_accuracy: 0.9200 - val_loss: 0.2434
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.9938 - loss: 0.0215 - val_accuracy: 0.9254 - val_loss: 0.2771
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - accuracy: 0.9955 - loss: 0.0157 - val_accuracy: 0.9160 - val_loss: 0.3557
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.9985 - loss: 0.0074 - val_accuracy: 0.9104 - val_loss: 0.3785
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - accuracy: 0.9980 - loss: 0.0076 - val_accuracy: 0.9137 - val_loss: 0.4373
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - accuracy: 0.9982 - loss: 0.0071 - val_ac

### Performance Metrics And Accuracy

In [102]:

y_pred1=np.argmax(model1.predict(X_test),axis=1)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step


In [108]:
y_pred1

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [103]:
from sklearn.metrics import confusion_matrix

In [104]:
confusion_matrix(y_test,y_pred1)

array([[3419,    0],
       [2616,    0]], dtype=int64)

In [105]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.5665285832642917

In [107]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



C:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
